In [ ]:
pip install datasets transformers==4.28.0 torch

  Using cached datasets-2.20.0-py3-none-any.whl (547 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 22.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 61.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 16.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 21.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.0 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cubl

In [ ]:
from datasets import load_dataset
from transformers import GPT2Tokenizer, GPT2LMHeadModel, DataCollatorForLanguageModeling, TrainingArguments, Trainer

In [ ]:
import torch

In [ ]:
dataset= load_dataset("wikipedia","20220301.en",split="train[:10000]")
eval_dataset= load_dataset("wikipedia","20220301.en",split="train[10000:11000]")

In [ ]:
dataset[0]

{'id': '12',
 'url': 'https://en.wikipedia.org/wiki/Anarchism',
 'title': 'Anarchism',
 'text': 'Anarchism is a political philosophy and movement that is sceptical of authority and rejects all involuntary, coercive forms of hierarchy. Anarchism calls for the abolition of the state, which it holds to be unnecessary, undesirable, and harmful. As a historically left-wing movement, placed on the farthest left of the political spectrum, it is usually described alongside communalism and libertarian Marxism as the libertarian wing (libertarian socialism) of the socialist movement, and has a strong historical association with anti-capitalism and socialism.\n\nHumans lived in societies without formal hierarchies long before the establishment of formal states, realms, or empires. With the rise of organised hierarchical bodies, scepticism toward authority also rose. Although traces of anarchist thought are found throughout history, modern anarchism emerged from the Enlightenment. During the latte

In [ ]:
model_name="gpt2"
model= GPT2LMHeadModel.from_pretrained(model_name)
tokenizer= GPT2Tokenizer.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [ ]:
tokenizer.pad_token=tokenizer.eos_token

In [ ]:
def tokenizer_function(examples):
  return tokenizer(examples["text"],truncation=True,padding="max_length",max_length=512)

In [ ]:
tokenizer_dataset= dataset.map(tokenizer_function,batched=True,remove_columns=["text"])
tokenizer_eval_dataset= eval_dataset.map(tokenizer_function,batched=True,remove_columns=["text"])

In [ ]:
training_args= TrainingArguments(
    output_dir="my_model",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    num_train_epochs=1,
    weight_decay=0.01
    )

In [ ]:
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

In [ ]:
trainer= Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenizer_dataset,
    eval_dataset=tokenizer_eval_dataset,
    data_collator=data_collator,
    tokenizer=tokenizer
)

In [ ]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,3.127600,2.990703


TrainOutput(global_step=5000, training_loss=3.1659199951171875, metrics={'train_runtime': 2076.4688, 'train_samples_per_second': 4.816, 'train_steps_per_second': 2.408, 'total_flos': 2612920320000000.0, 'train_loss': 3.1659199951171875, 'epoch': 1.0})

In [ ]:
trainer.save_model("./model1")
tokenizer.save_pretrained("./tokenizer1")

('./tokenizer1/tokenizer_config.json',
 './tokenizer1/special_tokens_map.json',
 './tokenizer1/vocab.json',
 './tokenizer1/merges.txt',
 './tokenizer1/added_tokens.json')

In [ ]:
tokenizer= GPT2Tokenizer.from_pretrained("./tokenizer1")
model= GPT2LMHeadModel.from_pretrained("./model1")

In [ ]:
input_text="once upon time"
inputs=tokenizer(input_text,return_tensors="pt")

In [ ]:
model.eval()

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [ ]:
with torch.no_grad():
  outputs=model.generate(inputs.input_ids,
                         max_length=512,
                         num_return_sequences=1,
                         pad_token_id=tokenizer.eos_token_id,
                         temperature=0.7,
                         top_p=0.9,
                         repetition_penalty=1.2
                         )

In [ ]:
generate_text= tokenizer.decode(outputs[0],skip_special_tokens=True)
print(generate_text)

once upon time, the term "disaster" is used to describe a sudden and catastrophic event that occurs in an area of significant economic importance.
The word disaster comes from Greek καρός (disaster), meaning "to happen", and δεινος (disaster), meaning 'a storm'. The first recorded instance of such events occurred on 9 January 1801 when Hurricane Isaac struck the island nation of Cyprus during its winter solstice celebrations. In addition it was also known as Katrina, the Great Flood, or Sandy.

History
In ancient Greece there were two major disasters: one caused by lightning strikes; another caused earthquakes; both resulted only after severe weather conditions had been encountered. Both catastrophes took place at different times throughout history.

The Greeks called these three storms "the storms of Athens". They are sometimes referred to collectively as cyclones. These storms have affected cities across Europe since antiquity, including Rome, Constantinople itself with devastating e

In [ ]:
!pwd

/content


In [ ]:
%cd /content/drive/MyDrive/wiki_dil_modeli

/content/drive/MyDrive/wiki_dil_modeli


In [ ]:
trainer.save_model("./model1")
tokenizer.save_pretrained("./tokenizer1")

('./tokenizer1/tokenizer_config.json',
 './tokenizer1/special_tokens_map.json',
 './tokenizer1/vocab.json',
 './tokenizer1/merges.txt',
 './tokenizer1/added_tokens.json')